# Aim

In step four, I'd like to address expanding the phylogeny with more R2R3 MYB subfamiles to address some open questions: 
* Can I add a subfamily label to the other Azfi sequences I found before with a custom hmm.
* Can I infer the phylogeny to more confidently/explicitly draw the Salvinales VII branch as a part of the remaining VII branch by including other subfamiles and an outgroup

In a later notebook, I'll address adding extra data like: 
Next steps: 

* Can I add intron/exon structures as a second line of evidence supporting the different subfamilies
* Finally, I'd like to log2 expressen/fold change to the Azfi sequences; to illustrate how these subfamilies change expression upon transistion to sexual reproduction at least in _Azolla_

Since retrieving sequences from the different databases is quite a pain, 
I'll first retrieve only NCBI listed sequences (which I can retrieve in batch).
Subsequently, I may reitterate in this notebook, expanding on that analysis with sequences from other genome databases.

Genes in new list files in the `data` directory were retrieved from NCBI. 
Acc. nr.s from the J&R study were mixed protein and gene id's, so batch retrieval wasn't possible.
Fasta files with both acc nr's were created in the same directory.

# prepare data

Combining and linearising fasta files.

In [6]:
for i in data/*fasta
do  inseq=$(echo $i | cut -d '/' -f 2 | cut -d '.' -f 1)
    cat data/$inseq.fasta \
     | awk '/^>/ {printf("%s%s\n",(N>0?"\n":""),$0);N++;next;} {printf("%s",$0);} END {printf("\n");}' \
     > data/"$inseq"_linear.fasta
done

In [3]:
ls data/*_linear.fasta

data/ARP_sequences_linear.fasta
data/Azfi-mybs-subfamVI-suspects_linear.fasta
data/Azfi-v1-MYB-sequences_linear.fasta
data/Azfi-v1-MYB-sequences_linear_linear.fasta
data/CDC5-outgroup_sequences_linear.fasta
data/combi_sequences_linear.fasta
data/combi_sequences_linear_linear.fasta
data/combi-VI-VII-Azfisuspects_linear.fasta
data/III_sequences_linear.fasta
data/II_sequences_linear.fasta
data/I_sequences_linear.fasta
data/IV_sequences_linear.fasta
data/MYB33_ARATH_linear.fasta
data/MYB33_ARATH_linear_linear.fasta
data/R1R2R3_sequences_linear.fasta
data/VIII_sequences_linear.fasta
data/VII_sequences_linear.fasta
data/VII_sequences_linear_linear.fasta
data/VI_sequences_linear.fasta
data/VI_sequences_linear_linear.fasta


In [7]:
cat data/CDC5-outgroup_sequences_linear.fasta \
    data/I_sequences_linear.fasta    \
    data/II_sequences_linear.fasta   \
    data/III_sequences_linear.fasta  \
    data/IV_sequences_linear.fasta   \
    data/V_sequences_linear.fasta    \
    data/VI_sequences_linear.fasta   \
    data/VII_sequences_linear.fasta  \
    data/VIII_sequences_linear.fasta \
    data/ARP_sequences_linear.fasta  \
    data/Azfi-mybs-subfamVI-suspects_linear.fasta \
    data/R1R2R3_sequences_linear.fasta \
    data/MYB33_ARATH_linear.fasta \
    > data/combi-I-to-VII-Azfi_linear.fasta
    